In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy
import matplotlib
import matplotlib.pyplot as plt
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import random
import sys
from IPython.display import display
from bs4 import BeautifulSoup
from IPython.display import display

In [2]:
me_rv_df = pd.DataFrame()
me_df = pd.DataFrame()
X = 'http://www.hostelworld.com/profile/18007308/reviews'
response = requests.get(X)
HTML = response.text
    
me_rv_df['text'] = Selector(text=HTML).xpath('//div[@id]/p/text()').extract()
me_rv_df['user_id'] = str(18007308)
me_rv_df['rev_date'] = Selector(text=HTML).xpath("//span[@class='reviewdate']/text()").extract()
me_rv_df['score'] = Selector(text=HTML).xpath("//div[@class='textrating']/text()").extract()
me_rv_df['hostel'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/text()").extract()
me_rv_df['location'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/span/text()").extract()
me_rv_df['link'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/@href").extract()

me_df['user_id'] = int(18007308)
me_df['about'] = [Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()[0]] 
me_df['num_revs']=len(me_rv_df['text']) 
me_df['user_id'] = int(18007308)

In [3]:
my_hostels_df=pd.DataFrame()
hostels_DNE = []

for i,url in enumerate(me_rv_df.link):

    hst_df = pd.DataFrame()
    X = str(url)
    response = requests.get(X)
    HTML = response.text
    
    free = Selector(text=HTML).xpath('//li[1]/div/p/text()').extract()
    general = Selector(text=HTML).xpath('//li[2]/div/p').extract()
    services = Selector(text=HTML).xpath('//li[3]/div/p').extract()
    food_drink = Selector(text=HTML).xpath('//li[4]/div/p').extract()
    entertainment = Selector(text=HTML).xpath('//li[5]/div/p').extract()
    policies = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()
    ratings = Selector(text=HTML).xpath("//div[@class='ratingbox']/text()").extract()
    description = Selector(text=HTML).xpath("//div[@class='section-propertytext']/div[@class='prop-intro']/div[@class='microdetailstext hiddentext prop-text bigtext ']/text()").extract()
    awards = len(Selector(text=HTML).xpath("//ul[@class='small-block-grid-1 medium-block-grid-2 large-block-grid-3']/li/b/text()").extract())
    num_rev = str(Selector(text=HTML).xpath("//div[@class='microsite_rating clearfix']/div/span[@class='reviews-count']/text()").extract()).split(" ")[0]
    rev_hl = Selector(text=HTML).xpath("//div[@class='small-12 medium-4 large-3 columns hide-for-small']/div/ul/li/span[@class='ratingfactorlabel']").extract()
    
    hst = me_rv_df.hostel.iloc[i]
    loc = me_rv_df.location.iloc[i]
    
    # prints out progress
#     if i%10 == 0:
#         sys.stdout.write('\r'+ str((float(i)/len(my_rv_df.link))*100)+'%')
        
    # if statements to make sure the link goes to a hostel that still exist
    if len(ratings) != 0:
        if float(ratings[0]) != 0.0:

            hst_df['rating'] = [float(ratings[0])]
            hst_df['desc'] = [description]
            hst_df['FREE'] = [free]
            hst_df['GENERAL'] = [general] 
            hst_df['SERVICES'] = [services]
            hst_df['FOOD_DRINK'] = [food_drink]
            hst_df['ENTERTAINMENT'] = [entertainment]
            hst_df['POLICIES'] = [policies]
            hst_df['hostel']= hst
            hst_df['location']= loc
            hst_df['num_revs'] = num_rev
            hst_df['num_awards'] = awards
            hst_df['rev_highlights'] = [rev_hl]
        
            my_hostels_df = my_hostels_df.append(hst_df,ignore_index=True)
            
        # else statment to collect index where hostles no longer exist
        else:
            hostels_DNE.append(i)
    # else statment to collect index where hostles no longer exist
    else:
        hostels_DNE.append(i)

In [57]:
# defs to remove tags and ugly things

def no_space_n(value):
    return str(value).replace('\n','').replace('  ','') 
def no_r(value):
    return str(value).replace('\r','') 
def no_n(value):
    return value.replace('\n','') 
def no_nc(value):
    return value.replace('\n,','').replace(',\n','') 
def add_c(value):
    return value.replace('  ',',')
def del_ec(value):
    return value.replace(',,,','').replace(',,',',')
def del_ec2(value):
    return value.replace(',,',',').replace(',,,',',').replace(',,',',')
def no_sc(value):
    return value.replace('.','').replace(',','').replace('!','').replace('?','').replace('*','').replace('-','').replace('_','').replace("'","").replace('/','').replace(':','').replace(";","")

def soupit(value):
    soup = BeautifulSoup(str(value))
    value = ''.join(soup.findAll(text=True))
    return value

def no_uni(value):
    return value.decode('unicode_escape').encode('ascii','ignore')

def rv_s(value):
    return value[1:]

In [58]:
my_hostels_df['FREE'][0].replace(' ','').replace('\\n','').replace('[','').replace(']','')

AttributeError: 'list' object has no attribute 'replace'

In [59]:
# clean hostel df

my_hostels_df['FREE'] = my_hostels_df['FREE'].map(no_space_n)
my_hostels_df['desc'] = my_hostels_df['desc'].map(no_space_n).map(no_r)
my_hostels_df['GENERAL'] = my_hostels_df['GENERAL'].map(soupit).map(add_c).map(del_ec).map(no_nc).map(no_uni)
my_hostels_df['SERVICES'] = my_hostels_df['SERVICES'].map(soupit).map(add_c).map(del_ec).map(no_nc) #.map(no_uni)
my_hostels_df['FOOD_DRINK'] = my_hostels_df['FOOD_DRINK'].map(soupit).map(add_c).map(del_ec).map(no_nc) #.map(no_uni)
my_hostels_df['ENTERTAINMENT'] = my_hostels_df['ENTERTAINMENT'].map(soupit).map(add_c).map(del_ec).map(no_nc).map(no_uni)
my_hostels_df['POLICIES'] = my_hostels_df['POLICIES'].map(soupit).map(add_c).map(del_ec2).map(no_nc).map(no_uni)
my_hostels_df['rev_highlights'] = my_hostels_df['rev_highlights'].map(soupit)

# split location
my_hostels_df['country'] = [a.split(",")[1] for a in my_hostels_df.location]
my_hostels_df['city'] = [a.split(",")[0] for a in my_hostels_df.location]
my_hostels_df.country = my_hostels_df.country.map(rv_s)

# make hostel desc lowercase and remove special characters
my_hostels_df.desc = [x.lower() for x in my_hostels_df.desc]
my_hostels_df['desc'] = my_hostels_df['desc'].map(no_sc)



/Users/caitlinmowdy/anaconda/envs/dsi/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [64]:
def no_sn(value):
    return value.replace('\\n','')

In [80]:
my_hostels_df['FREE'][2].split(',')[0].replace("[u'",'').replace(" '",'')

'Linen Included'

In [106]:
my_hostels_df['POLICIES']=my_hostels_df['POLICIES'].map(no_n)

In [116]:
def des_c(value):
    return value.replace('\\n','').replace('u\\r','').replace('u"\\r','')
my_hostels_df['desc'] = my_hostels_df['desc'].map(des_c)

In [117]:
my_hostels_df.head()

,rating,desc,FREE,GENERAL,SERVICES,FOOD_DRINK,ENTERTAINMENT,POLICIES,hostel,location,num_revs,num_awards,rev_highlights,country,city
0,9.0,[upariwana offers top quality affordable accom...,"[u'Free Breakfast ', u'Linen Included ', u'Fre...","[u',Security Lockers, Common Room, Book Exchan...","[u',Internet Access, Airport Transfers, Towels...","[u',Restaurant, Bar, Meals Available, Cafe, Te...","[u',DVD's, Board Games, Games Room, Foosball, ...","[u',Age Restriction, Credit Cards Accepted, No...",Pariwana Backpacker Hostels,"Lima, Peru",[],0,"[u'Perfect Location', u'Marvelous Staff', u'Fa...",Peru,Lima
1,8.7,[uwelcome to kipps hostel a friendly backpacke...,"[u'Linen Included ', u'Towels Included ', u'Fr...","[u',Security Lockers, Common Room, Bicycle Par...","[u',Luggage Storage, Tours/Travel Desk, 24 Hou...","[u',Mini-Supermarket, ']","[u',Pool Table, ']","[u',Age Restriction, Credit Cards Accepted, No...",Kipps Hostel Canterbury,"Canterbury, England",[],2,"[u'Wonderful Location', u'Marvelous Staff', u'...",England,Canterbury
2,8.9,[uour ecofriendly hostel is surrounded by gree...,"[u'Linen Included ', u'Free Parking ', u'Free ...","[u',Security Lockers, Common Room, Bicycle Par...","[u',Internet Access, Laundry Facilities, Bicyc...","[u',Tea & Coffee Making Facilities, ']","[u',DVD's, Wi-Fi, ']","[u',Child Friendly, No Curfew, Non Smoking,']",Tom's Comfort Hostel Waterford,"Waterford, Ireland",[],0,"[u'Great Location', u'Superb Staff', u'Fantast...",Ireland,Waterford
3,8.0,[uever stayed in a converted church then come ...,"[u'Linen Included ', u'Free City Maps ', u'Fre...","[u',Security Lockers, Common Room, BBQ, Breakf...","[u',Towels for hire, Luggage Storage, Tours/Tr...","[u',Bar, Vending Machines, Tea & Coffee Making...","[u',DVD's, Games Room, Pool Table, Wi-Fi, ']","[u',Age Restriction, Credit Cards Accepted, No...",Belford Hostel,"Edinburgh, Scotland",[],0,"[u'Great Location', u'Marvelous Staff', u'Awes...",Scotland,Edinburgh
